<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-2/Insumos/DeltaGamma_Integraci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"



/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 396 (delta 64), reused 0 (delta 0), pack-reused 311
Receiving objects: 100% (396/396), 1.51 MiB | 1.28 MiB/s, done.
Resolving deltas: 100% (247/247), done.


In [2]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12-0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 540698 bytes (528 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [0]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#interpolación
talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


# Definición de parámetros para valorar:
Acciones
Bonos
Bonos M
Bondes D
SWAPS
Futuros
Opciones

Cargamos parámetros para valoración:

In [0]:
%%R
fval=as.Date("20200228",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
#setwd(direc)	

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Symbols<-c  ("CEMEXCPO.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_eq=c(1000,2000,-600) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_fx=c(1000,-500,200) #monto inicial invertido en divisas
nh=3660 #días de historia

#CETES
base="RiesgosFinancieros/2020-2/Insumos/tasa_guber.txt"
plazos_bcc=cbind( 5,	18,	72,	115,	153,	245,	850,	1500,	3000,	6400,	7500,	9958)
contratos_bcc=cbind(22000, -29000, 10000, -46000, 44000, -26000,-30000,7000,-18000,-31000,23000,-23000)
nominal_bcc=10 #CETE

#BONOS M
btasadesc_bm="RiesgosFinancieros/2020-2/Insumos/tasa_yield.txt"
tfcupon_bm=cbind( 0.065,	0.0675,	0.07,	0.075,	0.078) #Tasafija del cupón
plazos_bm=cbind( 378,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon_bm=cbind( 182,	182, 182, 182, 182) #plazos_bm fijos de cada cupón
contratos_bm=cbind(22000, -29000, 29000, -46000, 10000) #posición invertida
nominal_bm=100

#BONDES D
btasadescst="RiesgosFinancieros/2020-2/Insumos/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2020-2/Insumos/tfondeo.txt"
plazos_bdm=cbind( 358,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon_bdm=cbind( 28,	28, 28, 28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm=cbind(220, -290, 290, -460, 100) #posición invertida
nominal_bdm=100

#FORWARDS TDC
bext="RiesgosFinancieros/2020-2/Insumos/tasa_libor.txt"
bdom="RiesgosFinancieros/2020-2/Insumos/tasa_fwd.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind( 360		)
contratos_fwd=cbind(500 )
kst_fwd=cbind(20.5 )
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind( 360)
contratos_fwd_ind=cbind(1)
kst_fwd_ind=cbind(52000)
nominal_fwd_ind=1


#SWAP
btasadesc_sw="RiesgosFinancieros/2020-2/Insumos/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2020-2/Insumos/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.079,0.075) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,360) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(-1600,1000) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1,1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,0) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

#OPCIONES
btasadesc_oir="RiesgosFinancieros/2020-2/Insumos/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2020-2/Insumos/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2020-2/Insumos/tvoltiie_opc.txt"
plazos_oir=cbind( 39,	115,	153,	245,	360) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0,0,0,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.07, 0.085, 0.078, 0.085, 0.08)
contratos_oir=cbind(2200, -2900, 2900, -460, 5000)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple



# Carga de datos

In [0]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_fx),t(pos_eq))
start_date=Sys.Date()-nh #fecha inicial
#Creación del objeto para guardar los datos
dataEnv<-new.env()
dataEnvFX<-new.env()
#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)
getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date)
#muestra
#tail(dataEnvFX$`GBPUSD=X`)
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(dataEnv,align='remove.na',fill.gaps = T)
bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
#muestra de datos
#head(dataEnv$prices)
#head(dataEnvFX$prices)
#Nos quedamos con los precios
stock_prices = dataEnv$prices
#tail(stock_prices[,])
stock_pricesFX=dataEnvFX$prices
#tail(stock_pricesFX)
#cambiar todo a pesos mexicanos
stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
#tail(stock_pricesFX)
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")
#stock_prices_EQFX
#tail(stock_prices_EQFX)
#Preciso actuales
#x0=as.data.table(as.matrix(stock_prices_EQFX[nrow(stock_prices_EQFX),])) #valores actuales
x0=stock_prices_EQFX[nrow(stock_prices_EQFX),]
#x0

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))


#LEER DATOS DE CETES #CARGA DE DATOS DE BONO CUPÓN CERO
data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

#LEER DATOS DE BONOS M
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc_bm)
n<-nrow(data1)
m_tybm=ncol(data1)
X_orig_tybm=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
X_orig_tybm=X_orig_tybm%>%select(-V1)
nodos_tybm=data.frame(data1[1,2:m_tybm])


##CARGA DE DATOS DE BONDE D
#carga de datos
#carga de tasas de descuento
data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
##X_orig_bd=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
#X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
#nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.

#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)

###Para Dolar

if (yext==1)
{
  #Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvFX<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
  
  #muestra de datos
  head(dataEnvFX$prices)
  
  #Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX$prices)),coredata(dataEnvFX$prices))
} else  
{  data3<-read.table("tasa_spot.txt")
n3<-nrow(data3)
m3<-ncol(data3)
X3=as.matrix(as.double(as.matrix(data3[2:(n+1),m3])))
}


# CARGA DE DATOS DE  FORWARD DE IPC
  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices)),coredata(dataEnvEQ$prices))

  
  #CARGA DE DATOS PARA SWAP
  data1<-read.table(btasadesc_sw)
  n1<-nrow(data1)
  m1_orig_sw=ncol(data1)
  X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X1_orig_sw=X1_orig_sw%>%select(-V1)
  nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
  data2<-read.table(btasacupvar_sw)
  n2<-nrow(data2)
  m2_orig_sw=ncol(data2)
  X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X2_orig_sw=X2_orig_sw%>%select(-V1)
  nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  
  
  #CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])



# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [33]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  
lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
lin_gub_bmy=data.table(Date=as.Date(lin_gub[X_orig_tybm,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber, con bonos M
lin_gub_bmybdst=data.table(Date=as.Date(lin_gub_bmy[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd
lin_gub_bmybdst_flibfwdsp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot
lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdsp[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index 
lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol

n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos

#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#historia de curva yield
X_orig_tybm=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X_orig_tybm,on=.(Date),nomatch=0][order(-Date)]
X_orig_tybm=X_orig_tybm%>%select(-Date)


#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)


#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#historia de swaps
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)



R[write to console]: Error in `[.data.table`(lin_gub_bmybdst_flibfwd, X3_ftdc, on = .(Date),  : 
  X3_ftdc is not found in calling scope and it is not a column name either. When the first argument inside DT[...] is a single symbol (e.g. DT[var]), data.table looks for var in calling scope.
Calls: <Anonymous> ... withVisible -> data.table -> as.Date -> [ -> [.data.table




Error in `[.data.table`(lin_gub_bmybdst_flibfwd, X3_ftdc, on = .(Date),  : 
  X3_ftdc is not found in calling scope and it is not a column name either. When the first argument inside DT[...] is a single symbol (e.g. DT[var]), data.table looks for var in calling scope.
Calls: <Anonymous> ... withVisible -> data.table -> as.Date -> [ -> [.data.table


# Pasos para medir el riesgo por el método de Delta-Gamma

Para todos los instrumentos hay que definir y calcular los siguientes elementos:
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Derivada con respecto al tiempo evaluada en el precio actual\
4.Primera derivada con respecto a factores de riesgo evaluada y multiplicada con su precio actual\
5.Segunda derivada con respecto a factores de riesgo evaluada y multiplicada con sus precios actuales (al cuadrado)\

## Acciones y Divisas



In [0]:
%%R
#divisas y acciones CÁLCULO

x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
mu_acc_div=0
g_acc_div=cbind(t(pos_fx),t(pos_eq))
g_acc_divt=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
gamma_acc_divt=matrix(0,ncol(x0_acc_div),ncol(x0_acc_div))


m_fx=length(pos_fx)
m_acc=length(pos_eq)

## CETES

In [27]:
%%R

#CETE CÁLCULO
#paso para interpolar las tasas
m=ncol(plazos_bcc)
x_bcc=matrix(0,n,m)
for (i in 1:(n))
{
  #x[i,]=approx(nodos_gov,x_orig_gov[i,],plazos_bcc)$y
  x_bcc[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],plazos_bcc,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],plazos_bcc)}
}

x0_bcc=x_bcc[1,]

#función
bonocupcero = function(i,t)
{
  1/(1+i*t/360)
}

#funcion primera derivada con tiempo
bcc_dt = function(i,t)
{
  -(i/360)/(1+i*t/360)^2
}

#funcion primera derivada con tasa interés
bcc_di = function(i,t)
{
  -(t/360)/(1+i*t/360)^2
}

#funcion segunda derivada con tasa de interés
bcc_di2 = function(i,t)
{
  2*(t/360)^2/(1+i*t/360)^3
}

V0_bcc=as.matrix(bonocupcero(x0_bcc,plazos_bcc))*contratos_bcc*nominal_bcc #Valor actual de cada bono

VT0_bcc=sum(V0_bcc) #Valor total del portafolio al tiempo 0

mu_bcc=bcc_dt(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
g_bcc=bcc_di(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
g_bcct=g_bcc*x0_bcc
gamma_bcc=bcc_di2(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
gamma_bcct2=diagv(gamma_bcc*(x0_bcc)^2)



R[write to console]: Error in xy.coords(x, y, setLab = FALSE) : 'x' and 'y' lengths differ
Calls: <Anonymous> ... withVisible -> approx -> regularize.values -> xy.coords




Error in xy.coords(x, y, setLab = FALSE) : 'x' and 'y' lengths differ
Calls: <Anonymous> ... withVisible -> approx -> regularize.values -> xy.coords


## Bonos M

In [28]:
%%R
#BONO M  CÁLCULO
#Posición inicial
#interpolación de tasas y volatilidades
m=ncol(plazos_bm)
X_bm=matrix(0,n,m)
for (i in 1:(n))
{
  X_bm[i,]=if(itpl==0){approx(nodos_tybm,X_orig_tybm[i,],plazos_bm,rule=2)$y}else{talamb(nodos_tybm,X_orig_tybm[i,],plazos_bm)}
}



#Función de valoración por tasa yield
bonoMyield=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
{
  N=as.integer(plazos_bm/plazocupon_bm)+1   
  a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
  p1=plazos_bm-plazocupon_bm*(N-1)   
  ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)   
}

x0_bm=t(as.matrix(X_bm[1,])) #tasas de descuento valor actual
V0_bm=bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)

#x=deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm")
#hs=as.expression(body(bonoMyield))
#deriv.formula(hs,"x")
#derivada errónea corregir
# dbonoMyield_t=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
# {
#   N=as.integer(plazos_bm/plazocupon_bm)+1   
#   a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
#   p1=plazos_bm-plazocupon_bm*(N-1)   
#   ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)*(x)^(3)  
# }

# #derivada errónea corregir
# dbonoMyield_x=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
# {
#   N=as.integer(plazos_bm/plazocupon_bm)+1   
#   a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
#   p1=plazos_bm-plazocupon_bm*(N-1)   
#   ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)*(plazos_bm/360)*x^(2)   
# }

# #derivada errónea corregir
# dbonoMyield_x2=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
# {
#   N=as.integer(plazos_bm/plazocupon_bm)+1   
#   a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
#   p1=plazos_bm-plazocupon_bm*(N-1)   
#   ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)*(plazos_bm/360)^2*x^(4)    
# }

#mu_bm=dbonoMyield_t(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)
mu_bm=Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "plazos_bm")
#g_bm=dbonoMyield_x(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)
g_bm=Deriv(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm"),"x0_bm")
g_bmt=g_bm*x0_bm
#gamma_bm=dbonoMyield_x2(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)
gamm_bm=Deriv(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm"),"x0_bm")
gamma_bmt2=diagv(gamma_bm*(x0_bm)^2)



R[write to console]: Error in xy.coords(x, y, setLab = FALSE) : 'x' and 'y' lengths differ
Calls: <Anonymous> ... withVisible -> approx -> regularize.values -> xy.coords




Error in xy.coords(x, y, setLab = FALSE) : 'x' and 'y' lengths differ
Calls: <Anonymous> ... withVisible -> approx -> regularize.values -> xy.coords


In [0]:
%%R
x=Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm")
x2=Deriv(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm"),"x0_bm")
hs=as.expression(body(bonoMyield))
#Deriv.formula(hs,"x")
#print(V0_bm)
x
x2

        [,1]     [,2]    [,3]      [,4]     [,5]
[1,] 3200876 -4730050 7870215 -47130960 20401546


## Bondes D

In [29]:
%%R
#BONDE D CÁLCULO

X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}


bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


V0_bdm=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)



d_bondeD_t=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((Xvp+Xst)/360)
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


#la derivada con respecto al valor presente y a la sobretasa es igual
d_bondeD_x=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360)
  V0f
} 


d_bondeD_tc=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  V0f=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  V0f
}



mgamma_bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }

  c11=matrix(0,1,length(contratos_bdmT)) #segunda derivada del cupón
  c12=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del cupón y primera del vp
  c13=c12 #primera del cupón y primera del st
  c21=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del vp y primera del cupon
  c22=2*((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^3))*nominal_bdm*((VTplazos_bdm)/360)^2 #segunda del vp
  c23=c22 #primera del vp y primera del st
  c31=c12 #primera del st y primera del cupon
  c32=c22 #primera del vp y primera del st
  c33=c22 #segunda del st
  rbind(cbind(diagv(c11),diagv(c12), diagv(c13)),cbind(diagv(c21),diagv(c22), diagv(c23)),cbind(diagv(c31),diagv(c32), diagv(c33)))
}


#mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))
Delta_X_bd_ext=X_bd_ext[1:(n-1),]/X_bd_ext[2:(n),]-1

mu_bd=d_bondeD_t(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
g_bd=cbind(d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv))
x0_bd_ext=cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,])))
g_bdt=g_bd*x0_bd_ext
gamma_bd=mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
gamma_bdt2=gamma_bd*(t(x0_bd_ext)%*%x0_bd_ext)
VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2)
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext)
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 
VaRTotalDGM=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(alpha) #VaR Normal
CVaRTotalDGM=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(alpha))/(1-alpha)

###VaR Cornish Fisher
tgammavar3_bdt=tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) 
asimPmargc_bdt=sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt=asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCF=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCF=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))

#Por factor de riesgo
VaRTotalDGMf=matrix(0,1,3)
CVaRTotalDGMf=matrix(0,1,3)
VaRTotalCFf=matrix(0,1,3)
CVaRTotalCFf=matrix(0,1,3)
 #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
x0_bd_extf=rbind(cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))))

for (j in (1:3))
{
g_bdt=g_bd*x0_bd_extf[j,]
gamma_bdt2=gamma_bd*((x0_bd_extf[j,])%*%t(x0_bd_extf[j,]))
VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2)
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext)
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 
VaRTotalDGMf[,j]=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(alpha) #VaR Normal
CVaRTotalDGMf[,j]=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(alpha))/(1-alpha)
###VaR Cornish Fisher
tgammavar3_bdt=tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) 
asimPmargc_bdt=sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt=asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCFf[,j]=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCFf[,j]=(sum(diag(tgammavar_bdt))/2+sum(mu_bd))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))
}
sum(V0_bdm)
VaRTotalDGM
CVaRTotalDGM
VaRTotalCF
CVaRTotalCF
VaRTotalDGMf
CVaRTotalDGMf
VaRTotalCFf
CVaRTotalCFf


##POR INSTRUMENTO

#Para obtener el VaR por contrato
VaRContDGM_bd=matrix(0,1,count(N_bd))
CVaRContDGM_bd=matrix(0,1,count(N_bd))
VaRContCF_bd=matrix(0,1,count(N_bd))
CVaRContCF_bd=matrix(0,1,count(N_bd))
#Para obtener el VaR por contrato por factor de riesgo
VaRContDGM_bdf=matrix(0,3,count(N_bd))
CVaRContDGM_bdf=matrix(0,3,count(N_bd))
VaRContCF_bdf=matrix(0,3,count(N_bd))
CVaRContCF_bdf=matrix(0,3,count(N_bd))

#V0=swap(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT, plazocuponT, VTplazos, Xvp[1,],N)


#Valor de las derivadas evaluadas en el punto x0.
V0f_dtc_bd=d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dvp_bd=d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dst_bd=V0f_dvp_bd

for (j in (1:count(N_bd)))
{
  if(j==1)
  { 
    x0cont_bd=c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])])
    #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
    x0cont_bdf=rbind(c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[1:sum(N_bd[1:j])],V0f_dvp_bd[1:sum(N_bd[1:j])],V0f_dst_bd[1:sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,1:sum(N_bd[1:j])],Xvp_bd[,1:sum(N_bd[1:j])],Xst_bd[,1:sum(N_bd[1:j])]))*(n-1)/(n-2)	
    gamma_cont_bd=gamma_bd[1:(3*sum(N_bd[1:j])),1:(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  else
  {
    x0cont_bd=c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])])
    x0cont_bdf=rbind(c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dvp_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dst_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))*(n-1)/(n-2)
    gamma_cont_bd=gamma_bd[(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j])),(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  
  
  tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
  tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
  VaRContDGM_bd[j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(alpha) #VaR Normal
  CVaRContDGM_bd[j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(alpha))/(1-alpha)
  ###VaR Cornish Fisher
  tgammavar3_bd=tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
  asimPmargc_bd=sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
  asimPmargc_bd=asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
  VaRmargcstd_bd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
  VaRContCF_bd[j]=(sum(diag(tgammavar_bd))/2+mucont_bd)+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
  CVaRContCF_bd[j]=(sum(diag(tgammavar_bd))/2+sum(mucont_bd))-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))
  for (k in (1:3))
  {
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bdf[k,]))
    tgamma_bd= x0cont_bdf[k,]%*%t(x0cont_bdf[k,])*gamma_cont_bd #gamma tilde
    tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
    tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
    VaRContDGM_bdf[k,j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(alpha) #VaR Normal
    CVaRContDGM_bdf[k,j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(alpha))/(1-alpha)
    ###VaR Cornish Fisher
    tgammavar3_bd=tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
    asimPmargc_bd=sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
    asimPmargc_bd=asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
    VaRmargcstd_bd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
    VaRContCF_bdf[k,j]=(sum(diag(tgammavar_bd))/2+mucont_bd)+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
    CVaRContCF_bdf[k,j]=(sum(diag(tgammavar_bd))/2+sum(mucont_bd))-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))    
  }
  
}
VaRContDGM_bd
CVaRContDGM_bd
VaRContCF_bd
CVaRContCF_bd
VaRContDGM_bdf
CVaRContDGM_bdf
VaRContCF_bdf
CVaRContCF_bdf



R[write to console]: Error in withVisible({ : 
  object 'lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in withVisible({ : 
  object 'lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible


## Forwards Tipo de Cambio

In [30]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)

for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd


dfuturoTC_tl = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=-s*((1+tn*t/360)/(1+tl*t/360)^2)*(t/360) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_tn = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  f1=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360)-(f1-k)/(1+t*tn/360)^2*(t/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_s = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  1/(1+t*tl/360) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_t = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn/360)/(1+tl*t/360))-s*((1+tn*t/360)/(1+tl*t/360)^2)*(tl/360) #Se obtiene el tipo de cambio forward
  #t(as.numeric((f)/(1+t*tn/360)-(f-k)/(1+t*tn/360)^2*(tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
  -(f-k)/(1+t*tn/360)^2*(tn/360)
}

mgamma_futTC= function(t,tl,tn,s,k,contratos_fwd,nominal_fwd)
{
c11= matrix(0,1,length(k))#spot spot
c12= -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd#spot tl
c13=  matrix(0,1,length(k))  #spot tn
c21=  -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl spot
c22= (2*s*((1+tn*t/360)/(1+tl*t/360)^3)*(t/360)^3)/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl tl
c23= matrix(0,1,length(k)) #tl tn
c31=(((t/360)/(1+tl*t/360))/(1+t*tn/360)-(((1+tn*t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn spot
c32=   (-s*((t/360)^2/(1+tl*t/360)^2)/(1+t*tn/360)-(s*((t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn tl
c33=  (s*((t/360)/(1+tl*t/360))/(1+t*tn/360)^2*(t/360)+2*(s*((1+tn*t/360)/(1+tl*t/360))-k)/(1+t*tn/360)^3*(t/360)^2-(s*((t/360)^2/(1+tl*t/360)))/(1+t*tn/360)^2)*contratos_fwd*nominal_fwd #tn tn
m=length(t)
rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}


X_futtdc=cbind(matrix(X3_ftdc,n,ncol(X1_fwtdc)),X1_fwtdc,X2_fwtdc)
xp_futtdc=t(as.matrix(X_futtdc[1,]))
mu_futtdc=dfuturoTC_t(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd
g_futtdc=cbind(dfuturoTC_s(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tl(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tn(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd)
g_futtdctT=xp_futtdc*g_futtdc
gamma_futtdc=mgamma_futTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd,contratos_fwd,nominal_fwd)
gamma_futtdctT2=gamma_futtdc*(t(xp_futtdc)%*%xp_futtdc)



R[write to console]: Error in xy.coords(x, y, setLab = FALSE) : 'x' and 'y' lengths differ
Calls: <Anonymous> ... withVisible -> approx -> regularize.values -> xy.coords




Error in xy.coords(x, y, setLab = FALSE) : 'x' and 'y' lengths differ
Calls: <Anonymous> ... withVisible -> approx -> regularize.values -> xy.coords


## Forwards Acciones o Índices

In [0]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m) #DIVIDENDOS
X2_fwind=matrix(0,n,m)

for (j in 1:n)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_find=as.matrix(X3_find)

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind

X_futind=cbind(matrix(X3_find,n,ncol(X1_fwind)),X1_fwind,X2_fwind)
xp_futind=t(as.matrix(X_futind[1,]))
mu_futind=dfuturoTC_t(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
g_futind=cbind(dfuturoTC_s(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tl(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tn(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind)
g_futindtT=xp_futind*g_futind
gamma_futind=mgamma_futTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind,contratos_fwd_ind,nominal_fwd_ind)
gamma_futindtT2=gamma_futind*(t(xp_futind)%*%xp_futind)



## SWAPS

In [0]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}


swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)


#derivada del swap con respecto aL tiempo
dswap_dtt=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*Xvp/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto a la tasa cupón variable
dswap_dtc=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  (((contratos_swT*plazocupon_swT)/360)/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto al valor presente
dswap_dvp=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

mgamma_swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  c11=matrix(0,1,ncol(contratos_swT))
  c21=-1*(((contratos_swT*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
  c12=-1*(((contratos_swT*plazocupon_swT*VTplazos_sw)/360^2)/(1+Xvp*VTplazos_sw/360)^2)*nominal_swT*(-1)^por_swT
  c22=2*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw^2/360^3)))/(1+Xvp*VTplazos_sw/360)^(3))*nominal_swT*(-1)^por_swT
  rbind(cbind(diagv(c11),diagv(c12)),cbind(diagv(c21),diagv(c22)))
}

X_sw=cbind( XtfwdT,Xvp)

mu_sw=sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))
g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]),dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))

#Multiplicación de la derivada por el punto x0
gt_sw=c( XtfwdT[1,],Xvp[1,])*g_sw

gamma_sw=mgamma_swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,])
tgamma_sw=c( XtfwdT[1,],Xvp[1,])%*%t(c( XtfwdT[1,],Xvp[1,]))*gamma_sw 


##Opciones de Tasa de Interés

In [0]:
%%R




##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades



opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio


#pr_oirIMERA DERIVADA

dopctintt = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial europea y TIEMPO
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(( -(S*dnorm(d1)*vol)/(2*sqrt(t/365))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/365)*(-1)^cp_oir)*(exp(-vp*t/365)))}else{(( -(S*dnorm(d1)*vol)/(2*sqrt(t/360))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/360)*(-1)^cp_oir)*(1/(1+vp*t/360)^2))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)


dopctintS = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2
}

#dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintvol = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintrho = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y rho
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)}else{-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)


mgammaopc=function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)	#matriz de segundas derivadas parciales de la call: rho, spot y vol de spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  c11=(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)^2}else{2*(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^3)*(-1)^cp_oir*(t/360)^2})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho con respecto a rho
  c21=(if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))*(-t/365)}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)^2))*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(-t/365)}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(-t/360)})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir #spot con respecto a rho
  c31=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol con respecto a rho
  c12=c21 #rho con respecto a spot
  c22=(if(cs_oir==1){(dnorm(d1)/(S*vol*sqrt(t/365)))*(exp(-vp*t/365))}else{(dnorm(d1)/(S*vol*sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) -2*((if(cs_oir==1){(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(exp(-vp*t/365)))*(-1)^cp_oir*(1/(S*vol*sqrt(t/365)))}else{(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(1/(1+vp*t/360)))*(1/(S*vol*sqrt(t/360)))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2) +2*(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^3/(1+S*pr_oir/dct_oir)^3*contratos_oir*nominal_oir #spot spot 
  c32=(if(cs_oir==1){dnorm(d1)*d2/vol*(exp(-vp*t/365))}else{dnorm(d1)*d2/vol*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) +   (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir # vol spot
  c13=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho vol
  c23=c32 #spot vol
  c33=(if(cs_oir==1){(dnorm(d1)*(S*sqrt(t/365)))*((d1*d2)/vol)*exp(-vp*t/365)}else{(dnorm(d1)*(S*sqrt(t/360)))*((d1*d2)/vol)*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol vol
  # outm=matrix(0,3,length(plazos_oir)*3)
  m=length(t)
 rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}

#mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)
X_oir=cbind(x1,x2,x3)
xp_oir=cbind(t(x01),t(x02),t(x03))
mu_oir=dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir 
delta_oir=cbind(dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir, dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir,dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir)
deltaT_oir=xp_oir*delta_oir
gammaT_oir=mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)
gammaT_oirT2=gammaT_oir*(t(xp_oir)%*%xp_oir)



# Integración de factores y cálculo de riesgo en conjunto

In [0]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,8,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(x_bcc) #cetes
n_if[3]=ncol(X_bm) #bonos m
n_if[4]=ncol(X_bd_ext) #bonde
n_if[5]=ncol(X_futtdc) #fut tdc
n_if[6]=ncol(X_futind) #fut ind
n_if[7]=ncol(X_sw) #swaps
n_if[8]=ncol(X_oir) #opciones tasa de interés

#valor del portafolios

V0_port=cbind(V0_bcc, V0_bm, V0_bdm, V0_fwtdc, V0_fwind, V0_sw, V0_oir)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,x_bcc,X_bm,X_bd_ext,X_futtdc,X_futind,X_sw,X_oir) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
VarDeltaX_port=as.matrix(cov(DeltaX_port)*(n-1)/(n-2))

mu_port=as.matrix(cbind(mu_acc_div,mu_bcc,mu_bm,mu_bd,mu_futtdc,mu_futind,mu_sw,mu_oir)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_futtdctT,g_futindtT,gt_sw,deltaT_oir)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_futtdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_futindtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2



#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
VaRT_port_DGM=(sum(diag(tgammavar_port))/2+sum(mu_port))-(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*qnorm(alpha) #VaR Normal
CVaRT_port_DGM=(sum(diag(tgammavar_port))/2+sum(mu_port))-(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*dnorm(qnorm(alpha))/(1-alpha)

tgammavar2_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]

###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF=(sum(diag(tgammavar_port))/2+sum(mu_port))+(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*VaRstd_port #VaR cf
CVaRT_port_DGM_CF=(sum(diag(tgammavar_port))/2+sum(mu_port))-(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*dnorm(VaRstd_port)/(pnorm(VaRstd_port))


VaRT_port_DGM/V0T_port
CVaRT_port_DGM/V0T_port
VaRT_port_DGM_CF/V0T_port
CVaRT_port_DGM_CF/V0T_port


            [,1]
[1,] 0.006832052
